## <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> PNL_S3: Generación de texto con RNN LSTM</p>

#### La generación de texto es un tipo de problema de modelado del lenguaje.

> 1. El modelado del lenguaje es el problema central para una serie de tareas de procesamiento del lenguaje natural, como voz a texto, sistemas conversacionales y el resumen de texto. 

> 2. Un modelo de lenguaje entrenado aprende la probabilidad de aparición de una palabra en función de la secuencia anterior de palabras utilizadas en el texto. 

> 3. Los modelos de lenguaje se pueden operar a nivel de carácter, nivel de n-grama, nivel de oración o incluso nivel de párrafo. 

**En este notebook:** se creará un modelo de lenguaje para generar texto en lenguaje natural implementando y entrenando una red neuronal recurrente del tipo LSTM.

#### Conjunto de datos:

> * Para este ejercicio utilizaremos el conjunto de datos [New York Times Comments and Headlines](https://www.kaggle.com/aashita/nyt-comments)el cual posee más de 9000 registros sobre noticias publicadas por este periódico entre 2017 y 2018. Entre las características de estas noticias encontramos su encabezado o headline.  

> * El objetivo de esta implementación será generar encabezados de noticias a partir del entrenamiento de una RNN del tipo LSTM que nos permita crear texto a partir de palabras claves. 

### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 1. Importar librerías</p>

In [2]:
# Propósito general
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import string, os 

# Librerías para crear y entrenar el modelo
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow.keras.utils as ku 

# Cálculo numérico y semillas aleatorias
import numpy as np
np.random.seed(7)
tf.random.set_seed(7) 
import random
random.seed(7)
tf.random.uniform([1], seed=1)

# Administrar advertencias 
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 2. Cargar el dataset</p>


> * En esta ocasión crearemos una función que nos permita extraer del dataset sólo los datos contenidos en la columna "headline", la cual es la información de interés. Para ello:

1. Disponemos del dataset en archivo .csv (NewYork_data.csv) en el ordenador 
2. Este dataset se encuentra en una carpeta denominada "NewYork" la cual está en la misma carpeta del notebook
3. Crearemos una función que busque en la carpeta definida: "NewYork", el archivo .csv del dataset  
4. La misma función extraerá del dataset los datos de la columna "headline" 
5. Con estos datos creará una lista de texto con la que trabajaremos en esta implementación: "all_headlines" 

In [3]:
# Crear objetos y función para cargar dataset con sólo los headlines de las noticias

# Definimos objeto curr_dir para definir la ruta de acceso a la carpeta del ordenador donde está el dataset "crudo"
curr_dir = './NewYork/'
# Definimos un objeto del tipo lista "all_headlines" para colectar en él los encabezados de las noticias del dataset
all_headlines = []

# Creamos función en búcle para buscar el dataset, extraer la info de la columna "headline" y crear una lista con ellos

# para cada archivo contenido en la carpeta "NewYork" (que está en la misma carpeta del notebook)
for filename in os.listdir(curr_dir):
    # si en la carpeta existe un archivo que contiene en su nombre las palabras "NewYork" 
    if 'NewYork' in filename:
        # Definir objeto para leer archivo .csv que cumpla con la condición de nombre establecido "NewYork"
        article_df = pd.read_csv(curr_dir + filename)
        # colectar información contenida en la columna "headline" del dataset en la lista creada "all_headlines"
        all_headlines.extend(list(article_df.headline.values))
        # finalizar función (queremos que solo lo haga con el dataset que cumpla con la condición de nombre "NewYork")
        break


# mostrar dimensión del dataset creada (lista)
print(len(all_headlines))

# mostrar primeros 10 registros del dataset creado (lista)
all_headlines[:10]

9335


['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan']

### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 3. Preprocesamiento de datos</p>


#### 3.1. Limpieza de datos

* Realizaremos una limpieza de los datos de texto colectados. Esto incluye crear una función que:

1. Eliminar signos de puntuación 
2. Convertir a minúsculas

In [4]:
# Definimos función para elimnar puntuaciones y convertir a minúsculas

# definimos función "clean_text" para archivo de texto
def clean_text(txt):
    # objeto "txt" para remover signos de puntuación y convertir a minúscula 
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    # codificar la cadena de texto (string) en formato UTF-8 y decodificarlo en formato Ascii
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

# aplicamos la función creada al corpus 
corpus = [clean_text(x) for x in all_headlines]

# Mostrar primeros diez registros del corpus procesado
corpus[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

#### 3.2. Tokenización: tokens tipo Ngram 

* El modelado del lenguaje requiere una secuencia de datos de entrada, ya que dada una secuencia (de palabras/tokens), el objetivo es predecir la siguiente palabra/token.

* La tokenización es un proceso de extracción de tokens (términos/palabras) de un corpus. 

* La biblioteca Keras de Python tiene un modelo incorporado para la tokenización que se puede usar para obtener los tokens y su índice en el corpus. Esto dará como resultado una secuencia de tokens creados a partir de la fragmentación del corpus. 


In [5]:
# definimos función "get_sequence_of_tokens" para aplicar al corpus
def get_sequence_of_tokens(corpus):
    # aplicamos la función ".fit_to_texts" de TF para aplicar al corpus
    tokenizer.fit_on_texts(corpus)
    # almacenamos en "total_words" los índices de los tokens
    total_words = len(tokenizer.word_index) + 1
    
    # Convertir los datos en una secuencia etiquetada
    input_sequences = []
    # para cada línea en el corpus
    for line in corpus:
        # objeto para aplicar función .text_to_sequences de TF a cada línea del corpus
        token_list = tokenizer.texts_to_sequences([line])[0]
        # función búcle para cada token 
        for i in range(1, len(token_list)):
            # establecer token tipo Ngram
            n_gram_sequence = token_list[:i+1]
            # Almacenar Ngramas creados
            input_sequences.append(n_gram_sequence)
    # como resultado darán las secuencias de entrada (tokens) y sus índices
    return input_sequences, total_words

# aplicamos la función creada al corpus para su tokenización en Ngramas
tokenizer = Tokenizer()
inp_sequences, total_words = get_sequence_of_tokens(corpus)

# mostrar secuencias creadas
inp_sequences[:10]

[[392, 18],
 [392, 18, 5167],
 [392, 18, 5167, 524],
 [392, 18, 5167, 524, 4],
 [392, 18, 5167, 524, 4, 2],
 [392, 18, 5167, 524, 4, 2, 1602],
 [392, 18, 5167, 524, 4, 2, 1602, 135],
 [392, 18, 5167, 524, 4, 2, 1602, 135, 5],
 [392, 18, 5167, 524, 4, 2, 1602, 135, 5, 1952],
 [7, 58]]


1. En la salida de la celda anterior vemos datos del tipo [392,18], [392,18,5176], [392,18,5176,524] etc. 
2. Estos representan las frases ngram generadas a partir de los datos de entrada, donde cada número entero corresponde al índice de una palabra en particular en el vocabulario completo presente en el texto. Por ejemplo

**Encabezado:** i stand  with the shedevils  
**Ngrams:** | Sequencia de Tokens

<table>
<tr><td>Ngram </td><td> Sequencia de Tokens</td></tr>
<tr> <td>i stand </td><td> [392,18] </td></tr>
<tr> <td>i stand with </td><td> [392,18,5176] </td></tr>
<tr> <td>i stand with the </td><td> [392,18,5176,524] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [392,18,5176,524,4] </td></tr>
</table>


#### 3.3. Completar secuencias y obtener variables: predictors y label

* Ahora que hemos generado un conjunto de datos que contiene secuencias de tokens, las diferentes secuencias pueden tener diferentes longitudes. 
* Antes de comenzar a entrenar el modelo, necesitamos rellenar las secuencias y hacer que tengan la misma longitud. 

Para esto, 

1. Usaremos la función pad_sequence de Kears. 
2. Para introducir estos datos en un modelo de aprendizaje, necesitamos crear predictores y etiquetas. 
3. Crearemos una secuencia de N-gramas como predictor y la siguiente palabra del N-grama como etiqueta. Por ejemplo:


Encabezado:  i stand with the shedevils

<table>
<tr><td>PREDICTORES</td> <td>           ETIQUETA </td></tr>
<tr><td>i                   </td> <td>  stand</td></tr>
<tr><td>i stand               </td> <td>  with</td></tr>
<tr><td>i stand with      </td> <td>  the</td></tr>
<tr><td>i stand with the </td> <td>  shedevil</td></tr>
</table>

In [6]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 4. Crear modelo</p>


## Redes Neuronales Recurrentes LSTM

![](http://www.shivambansal.com/blog/text-lstm/2.png)

* A diferencia de las redes neuronales Feed-forward en las que las salidas de activación se propagan solo en una dirección, en las RNN las salidas de activación de las neuronas se propagan en ambas direcciones (de entradas a salidas y de salidas a entradas). 

* Esto crea bucles en la arquitectura de la red neuronal que actúa como un "estado de memoria" de las neuronas.Este estado permite a las neuronas recordar lo que se ha aprendido hasta el momento.

* El estado de la memoria en las RNN ofrece una ventaja sobre las redes neuronales tradicionales, pero se asocia con ellas un problema llamado gradiente de fuga. 

* En este problema, mientras se aprende con una gran cantidad de capas, se vuelve muy difícil para la red aprender y ajustar los parámetros de las capas anteriores Para abordar este problema, se ha desarrollado un nuevo tipo de RNN llamados Modelos LSTM (Memoria a largo plazo corto).

* Los LSTM tienen un estado adicional llamado "estado de celda" a través del cual la red realiza ajustes en el flujo de información. La ventaja de este estado es que el modelo puede recordar u olvidar las inclinaciones de manera más selectiva.  

#### Vamos a diseñar un modelo LSTM:

> 1. Capa de entrada: toma la secuencia de palabras como entrada
> 2. Capa LSTM: calcula la salida usando unidades LSTM. He agregado 100 unidades en la capa, pero este número se puede ajustar más tarde.
> 3. Capa de abandono: una capa de regularización que apaga aleatoriamente las activaciones de algunas neuronas en la capa LSTM. Ayuda a prevenir el sobreajuste. (Capa opcional)
> 4. Capa de salida: calcula la probabilidad de la mejor palabra siguiente posible como salida

In [7]:
# Para crear el modelo definiremos una función 

# definimos función "create_model" max sequence y total words
def create_model(max_sequence_len, total_words):
    # Dimensión de la entrada
    input_len = max_sequence_len - 1
    # tipo del modelo 
    model = Sequential()
    
    # adicionamos capa tipo Embedding)
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # adicionamos capa tipo LSTM
    model.add(LSTM(100))
    # adicionamos capa de dropout
    model.add(Dropout(0.1))
    
    # adicionamos capa Densa de salida
    model.add(Dense(total_words, activation='softmax'))
    
    # definimos parametros de .compile para el modelo
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = 'accuracy')
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   
                                                                 
Total params: 1294815 (4.94 MB)
Trainable params: 1294815 (4.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 5. Entrenar modelo</p>


In [8]:
# entrenaremos nuestro modelo con 100 épocas 
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
1618/1618 [==============================] - 62s 37ms/step - loss: 7.8411 - accuracy: 0.0365
Epoch 2/100
1618/1618 [==============================] - 68s 42ms/step - loss: 7.4359 - accuracy: 0.0485
Epoch 3/100
1618/1618 [==============================] - 73s 45ms/step - loss: 7.2529 - accuracy: 0.0571
Epoch 4/100
1618/1618 [==============================] - 78s 48ms/step - loss: 7.0510 - accuracy: 0.0651
Epoch 5/100
1618/1618 [==============================] - 78s 48ms/step - loss: 6.8440 - accuracy: 0.0744
Epoch 6/100
1618/1618 [==============================] - 82s 51ms/step - loss: 6.6235 - accuracy: 0.0802
Epoch 7/100
1618/1618 [==============================] - 83s 51ms/step - loss: 6.3966 - accuracy: 0.0881
Epoch 8/100
1618/1618 [==============================] - 82s 51ms/step - loss: 6.1649 - accuracy: 0.0936
Epoch 9/100
1618/1618 [==============================] - 83s 52ms/step - loss: 5.9465 - accuracy: 0.1019
Epoch 10/100
1618/1618 [==============================]

1618/1618 [==============================] - 77s 48ms/step - loss: 1.7831 - accuracy: 0.6170
Epoch 79/100
1618/1618 [==============================] - 78s 48ms/step - loss: 1.7753 - accuracy: 0.6177
Epoch 80/100
1618/1618 [==============================] - 81s 50ms/step - loss: 1.7679 - accuracy: 0.6194
Epoch 81/100
1618/1618 [==============================] - 74s 46ms/step - loss: 1.7599 - accuracy: 0.6184
Epoch 82/100
1618/1618 [==============================] - 74s 46ms/step - loss: 1.7565 - accuracy: 0.6180
Epoch 83/100
1618/1618 [==============================] - 75s 46ms/step - loss: 1.7414 - accuracy: 0.6226
Epoch 84/100
1618/1618 [==============================] - 75s 46ms/step - loss: 1.7380 - accuracy: 0.6234
Epoch 85/100
1618/1618 [==============================] - 73s 45ms/step - loss: 1.7321 - accuracy: 0.6242
Epoch 86/100
1618/1618 [==============================] - 78s 49ms/step - loss: 1.7166 - accuracy: 0.6266
Epoch 87/100
1618/1618 [==============================] - 8

### <p style="background-color:Lime;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> 6. Generar texto</p>

Para generar texto:

1. Crearemos una función que prediga la siguiente palabra en función de la palabra de entrada (o texto semilla). 
2. Tokenizaremos el texto semilla, completaremos la secuencia y la pasaremos al modelo entrenado para obtener las palabras predichas. 
3. Se pueden sumar varias palabras predichas para obtener la secuencia predicha.

In [9]:
# crear función 
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted, axis=1)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

#### 6.1. Ver resultados de predicción 

In [10]:
print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("united states", 10, model, max_sequence_len))
print (generate_text("united states", 15, model, max_sequence_len))

United States And Good Or Becomes A
United States And Good Or Becomes A White House To Those Is
United States And Good Or Becomes A White House To Those Is Out To A Famous News


In [11]:
print (generate_text("president trump", 3, model, max_sequence_len))
print (generate_text("president trump", 4, model, max_sequence_len))
print (generate_text("president trump", 5, model, max_sequence_len))
print (generate_text("president trump", 8, model, max_sequence_len))


President Trump Is In The
President Trump Is In The Building
President Trump Is In The Building In
President Trump Is In The Building In America Is The


In [12]:
print (generate_text("joe biden", 3, model, max_sequence_len))
print (generate_text("joe biden", 4, model, max_sequence_len))
print (generate_text("joe biden", 5, model, max_sequence_len))
print (generate_text("joe biden", 8, model, max_sequence_len))

Joe Biden Still Be Access
Joe Biden Still Be Access And
Joe Biden Still Be Access And A
Joe Biden Still Be Access And A Key Time On


In [13]:
print (generate_text("india and china", 3, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("india and china", 5, model, max_sequence_len))
print (generate_text("india and china", 8, model, max_sequence_len))

India And China A Changemaker Nights
India And China A Changemaker Nights Dont
India And China A Changemaker Nights Dont Be
India And China A Changemaker Nights Dont Be Far To Write


In [14]:
print (generate_text("european union", 3, model, max_sequence_len))
print (generate_text("european union", 4, model, max_sequence_len))
print (generate_text("european union", 5, model, max_sequence_len))
print (generate_text("european union", 8, model, max_sequence_len))

European Union Two Blocked In
European Union Two Blocked In Anthony
European Union Two Blocked In Anthony To
European Union Two Blocked In Anthony To Fall A Divorce
